<a href="https://colab.research.google.com/github/sandroormeno/TUTORIAL-TENSORFLOW-LITE-ARDUINO-DUE/blob/master/monitor_exercice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import pandas
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
print(tf.__version__)

2.0.0-rc2


In [0]:
# Los datos
url  = 'https://raw.githubusercontent.com/sandroormeno/Neural-network-con-arduino-ATmega644/master/jupyter%20notebook/datos_aceler%C3%B3metro.csv'

In [0]:
datos = pandas.read_csv(url)
datos[:30]

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,y4,z4,x5,y5,z5,clase
0,141,127,132,133,120,147,170,114,120,132,120,129,143,123,142,0
1,159,122,132,135,120,130,146,129,140,159,116,134,132,122,126,0
2,148,124,144,159,125,136,135,123,126,149,125,139,155,121,137,0
3,133,124,128,149,123,144,156,124,137,134,123,126,147,125,140,0
4,161,118,131,133,120,127,148,122,144,162,124,131,134,125,125,0
5,142,126,137,167,113,133,136,122,127,138,128,135,169,120,132,0
6,138,125,128,133,123,131,181,114,142,137,120,128,130,123,128,0
7,178,111,139,142,125,129,130,123,127,170,126,143,143,122,137,0
8,129,122,127,164,119,150,150,125,136,131,124,127,152,125,144,0
9,152,124,140,134,123,127,146,125,142,160,124,137,136,124,128,0


In [0]:
# Las 15 primeras columnas son los características
caracteristicas= datos.values[:,:15]
# La última columna se coonvierte en los etiquetas
etiquetas= datos.values[:,15]

In [0]:
# Normalizamos las carasterísticas. Las dividimos entre 255
caracteristicas_norm= caracteristicas/255

In [0]:
# Codificación en One hot
etiquetas__hot = to_categorical(etiquetas)

In [0]:
caracteristicas.shape

In [0]:
# Dividimos los datos en estrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(caracteristicas_norm, etiquetas__hot, test_size=0.2, random_state=21)

In [0]:
# El modelo
model= tf.keras.Sequential()

model.add(layers.Dense(16, activation='relu', input_dim=15))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(3, activation='sigmoid'))

# Configulación del entrenamiento
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
# El entrenamiento
model.fit(X_train, y_train, epochs=300, batch_size=5)

In [0]:
# La evaluación
score = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [0]:
# aquí termina el 1er tutorial

In [0]:
# Stay here

# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("exercice_model.tflite", "wb").write(tflite_model)

In [0]:
X_test = X_test.astype('float32')
#print len(X_test) 

# Instantiate an interpreter for each model
exercice_model = tf.lite.Interpreter('exercice_model.tflite')

# Allocate memory for each model
exercice_model.allocate_tensors()

# Get the input and output tensors so we can feed in values and get the results
input_details = exercice_model.get_input_details()
output_details = exercice_model.get_output_details()


print('[          Predicción         ]' + "       " + '[  Real  ]' + "\n")
for i in range(len(X_test)):

  # Run each model's interpreter for each value and store the results in arrays
  input_shape = input_details[0]['shape']
  input_data = X_test[i].reshape((1,15))
  exercice_model.set_tensor(input_details[0]['index'],input_data )

  exercice_model.invoke()
  output_data = exercice_model.get_tensor(output_details[0]['index'])
    
  if (np.argmax(output_data) == np.argmax(y_test[i])):
    print("[ %.5f,  %.5f,  %.5f ]   =   %s " %  (output_data[0][0], output_data[0][1], output_data[0][2],  str(y_test[i])))
  else:
    
    print("[ %.5f,  %.5f,  %.5f ]  <>   %s !!!!" %  (output_data[0][0], output_data[0][1], output_data[0][2],  str(y_test[i])))
  
 
  #print np.argmax(output_data)
  #print(output_data)

[          Predicción         ]       [  Real  ]

[ 0.18411,  0.00092,  0.00000 ]   =   [1. 0. 0.] 
[ 0.00000,  0.00101,  0.00000 ]   =   [0. 1. 0.] 
[ 0.65057,  0.00118,  0.00000 ]   =   [1. 0. 0.] 
[ 0.03930,  0.00133,  0.00000 ]   =   [1. 0. 0.] 
[ 0.00000,  0.00179,  0.00000 ]   =   [0. 1. 0.] 
[ 0.00000,  0.00003,  0.99766 ]   =   [0. 0. 1.] 
[ 0.00414,  0.00067,  0.00000 ]   =   [1. 0. 0.] 
[ 0.00001,  0.00088,  0.00000 ]   =   [0. 1. 0.] 
[ 0.00000,  0.00001,  1.00000 ]   =   [0. 0. 1.] 
[ 0.50828,  0.00080,  0.00000 ]   =   [1. 0. 0.] 
[ 0.00000,  0.00034,  0.00125 ]   =   [0. 0. 1.] 
[ 0.00000,  0.00006,  0.34446 ]   =   [0. 0. 1.] 
[ 0.00000,  0.00002,  0.99900 ]   =   [0. 0. 1.] 
[ 0.59492,  0.00105,  0.00000 ]   =   [1. 0. 0.] 
[ 0.75483,  0.00099,  0.00000 ]   =   [1. 0. 0.] 
[ 0.00000,  0.00041,  0.00378 ]   =   [0. 0. 1.] 
[ 0.00000,  0.00041,  0.00029 ]  <>   [0. 0. 1.] !!!!
[ 0.79026,  0.00076,  0.00000 ]   =   [1. 0. 0.] 
[ 0.00000,  0.00009,  0.16312 ]   =   [0. 0. 1

In [0]:
# aquí termina el 2do tutorial

In [0]:
import os
basic_model_size = os.path.getsize("exercice_model.tflite")
print("Basic model is %d bytes" % basic_model_size)

Basic model is 3856 bytes


In [0]:
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i exercice_model.tflite > exercice_model.cc
# Print the source file
!cat exercice_model.cc

Selecting previously unselected package xxd.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../xxd_2%3a8.0.1453-1ubuntu1.1_amd64.deb ...
Unpacking xxd (2:8.0.1453-1ubuntu1.1) ...
Setting up xxd (2:8.0.1453-1ubuntu1.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
unsigned char exercice_model_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x12, 0x00,
  0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00,
  0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0xbc, 0x0e, 0x00, 0x00, 0x10, 0x0a, 0x00, 0x00, 0xf8, 0x09, 0x00, 0x00,
  0x34, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0xe8, 0xf1, 0xff, 0xff, 0x08, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f,
  0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72, 0x73,
  0x69, 0x6f, 0x6e, 0x00, 0x0d, 0x00, 0x00,